<h1 align="center">Analysis of the Amazon dataset of book reviews</h1>

The aim was to process the semantic analysis of Ukrainian texts, so the collection of positive and negative reviews was translated into Ukrainian using Google Translate.

To get started, we import all the libraries we need:

In [1]:
import os
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import xml.etree.ElementTree as ET

%matplotlib inline

Next, since our reviews are in XML format, we need to process each line to get the value from the tags.

**Note**: 
* `errors='ignore'`used to skip all possible incorrect characters in our documents;

* `readLines` return all lines in the file, as a list where each line is an item in the list object.

In [2]:
testRead = open('data/positive.review', 'r', encoding='utf8', errors='ignore').readlines()
for i in range(84):
    print(testRead[i])

<review>

<unique_id>

0375416811: owner_of_chick_lit: erin_oakes_ "avid_reader"

</unique_id>

<unique_id>

1521 рік

</unique_id>

<asin>

0375416811

</asin>

<product_name>

Щоденник Бріджит Джонс: Книги: Хелен Філдінг, Трейсі Беннет

</product_name>

<product_type>

книги

</product_type>

<product_type>

книги

</product_type>

<helpful>



</helpful>

<rating>

5.0

</rating>

<title>

Засновник Чик-літ

</title>

<date>

24 серпня 2006 р

</date>

<reviewer>

Ерін Оукс "завзятий читач"

</reviewer>

<reviewer_location>

Центральна ПА

</reviewer_location>

<review_text>

Бріджит Джонс, сучасна жінка, блискуча і не знає цього, схильна до нещасних випадків та безладів, але їм вдається вийти з них.



Це книга, яка розпочала все з лихоманки, що запалюється. Бріджит Джонс - мій герой!

</review_text>

</review>

<review>

<unique_id>

188105201X: відмінний_ресурс_для_принципів!: Onickre@mail.milwaukee.k12.wi.us

</unique_id>

<unique_id>

3294

</unique_id>

<asin>

188105201X

</a

In our directory with training sets we have two files - with positive and negative reviews, each of these files contains 1000 unprocessed reviews. Let's check this:

In [3]:
pos_reviews = open('data/positive.review', 'r', encoding='utf8', errors='ignore').readlines()
neg_reviews = open('data/negative.review', 'r', encoding='utf8', errors='ignore').readlines()
print("Number of Positive Reviews: ", pos_reviews.count('</review>\n'), 
      "\nNumber of Negative Reviews: ", neg_reviews.count('</review>\n'))

Number of Positive Reviews:  1000 
Number of Negative Reviews:  1000


<h2 align="center">XML Parse</h2>

Now we will try to parse our XML file at a basic level. We will create a dictionary from the lists. Each list will contain certain tags and their content for single review.

In [7]:
def parse_xml(xml_reviews):
    review, reviews = [], {}
    count = 0
    
    for i in range(len(xml_reviews)):
        if xml_reviews[i] != '</review>\n':
            if xml_reviews[i] == '<review>\n' and xml_reviews[i+1] == '<unique_id>\n':
                review.append('unique_id | ' + xml_reviews[i+2])  # unique_id
            
            if  xml_reviews[i] == '<asin>\n':
                review.append('asin | ' + xml_reviews[i+1]) # asin   
            
            if  xml_reviews[i] == '<product_name>\n':
                review.append('product_name | ' + xml_reviews[i+1]) # product_name
                
            if xml_reviews[i] == '<helpful>\n':
                review.append('helpful | ' + xml_reviews[i+1]) # helpful
            
            if xml_reviews[i] == '<rating>\n':
                review.append('rating | ' + xml_reviews[i+1]) # rating
            
            if xml_reviews[i] == '<title>\n':
                review.append('title | ' + xml_reviews[i+1]) # title
            
            if xml_reviews[i] == '<date>\n':
                review.append('date | ' + xml_reviews[i+1]) # date
            
            if xml_reviews[i] == '<reviewer>\n':
                review.append('reviewer | ' + xml_reviews[i+1]) # reviewer
            
            if xml_reviews[i] == '<reviewer_location>\n':
                review.append('reviewer_location | ' + xml_reviews[i+1]) # reviewer_location
            
            if xml_reviews[i] == '<review_text>\n':
                review.append('review_text | ' + xml_reviews[i+1]) # review_text
        
        elif xml_reviews[i] == '</review>\n':
            count += 1
            rev_name = 'review'+ str(count) 
            reviews[rev_name] = review
            review.clear()
    return reviews

In [8]:
posReviewsDict = parse_xml(pos_reviews)
negReviewsDict = parse_xml(neg_reviews)
posReviewsDict['review1']

['unique_id | 1934166332: ковбойська_любов: к._люди\n',
 'asin | 1934166332\n',
 'product_name | Під цим ковбойським капелюхом: Книги: Роб, Найт\n',
 'helpful | \n',
 'rating | 4.0\n',
 'title | Ковбойське кохання\n',
 'date | 4 квітня 2007 р\n',
 'reviewer | К. Народи\n',
 'reviewer_location | Арлінгтон, штат Вірджинія\n',
 'review_text | Я хотів би дати цій книзі чотири з половиною зірки замість чотирьох; Я не можу цілком виправдати п\'ять зірок у своєму розумі. Дві історії були безумовно хорошими: перша, "Вершники в масках" Пархеліона, і третя "Рікоше" Б.А. Тортуги. Мені сподобалося і те, і інше, але я не був особливо засніжений інтенсивністю конфлікту / лінії сюжету чи головними героями та їх стосунками. Не така друга історія: "Повішений" від Кет Кейн був потужною, напруженою та зворушливою історією про двох чудових, але дефектних персонажів, у яких були секрети, що розривали їхні стосунки, поки вони не змогли (окремо) працювати над проблемами. Ця історія заслужила п’ять зірок плюс